#Dummy data
Add some dummy data that we will clean up

In [0]:
use usertraining.tpch;

insert into customer_bronze
  select *
  from customer_bronze
  limit 10;

insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;
insert into customer_bronze select null, null, null, null, null, null, null, null, null;

In [0]:
show create table customer_bronze

#Create Silver table

In [0]:
CREATE TABLE IF NOT EXISTS customer_silver
(
  c_custkey BIGINT,
  c_name STRING,
  c_address STRING,
  c_nationkey BIGINT,
  c_phone STRING,
  c_acctbal DECIMAL(18,2),
  c_mktsegment STRING,
  c_comment STRING,
  updated TIMESTAMP,
  updated_date DATE,
  updated_time STRING,
  phone_area STRING
);

In [0]:
CREATE OR REPLACE TABLE customer_silver_working AS
  SELECT * FROM customer_bronze;

SELECT * FROM customer_silver_working;

#Data Profile
Click the + symbol and select Data Profile.

In [0]:
SELECT * FROM customer_silver_working;

Databricks data profile. Run in Databricks to view.

#Missing data

In [0]:
select * from customer_silver_working where c_custkey is null

In [0]:
CREATE OR REPLACE TABLE customer_silver_working AS
  SELECT * FROM customer_bronze WHERE c_custkey IS NOT NULL;

SELECT * FROM customer_silver_working;

#Deduplicate rows

In [0]:
SELECT count(*) FROM customer_silver_working

In [0]:
INSERT OVERWRITE customer_silver_working
  SELECT DISTINCT(*) FROM customer_silver_working

#Deduplicate rows based on specific column
Use max as a hack to keep the other cols, but groups by of custkey

In [0]:
INSERT OVERWRITE customer_silver_working
  SELECT
    c_custkey,
    max(c_name),
    max(c_address),
    max(c_nationkey),
    max(c_phone),
    max(c_acctbal),
    max(c_mktsegment),
    max(c_comment),
    max(updated)
  FROM customer_silver_working
  WHERE c_custkey IS NOT NULL
  GROUP BY c_custkey;

SELECT count(*) FROM customer_silver_working

In [0]:
SELECT * FROM customer_silver_working

#Validate dateset
Validate custkey is unique

In [0]:
SELECT max(row_count) <= 1 no_duplicate_ids FROM (
  SELECT c_custkey, COUNT(*) as row_count
  FROM customer_silver_working
  GROUP BY c_custkey
)

confirm each custkey is uniqiue to a c_address

In [0]:
SELECT max(c_custkey_count) <= 1 most_c_custkey FROM (
  SELECT c_address, COUNT(c_custkey) as c_custkey_count
  FROM customer_silver_working
  WHERE c_address IS NOT NULL
  GROUP BY c_address
)

#Date format & regex

In [0]:
INSERT INTO customer_silver
(
  SELECT
    *,
    to_date(updated) updated_date,
    date_format(updated, "HH:mm:ss") updated_time,
    regexp_extract(c_phone, "(\\d+?)-", 1) phone_area
  FROM customer_silver_working
);

In [0]:
SELECT * FROM customer_silver